In [2]:
import warnings
warnings.filterwarnings("ignore")

from scipy import constants as cst
from pint import UnitRegistry
ureg = UnitRegistry()
ureg = UnitRegistry(autoconvert_offset_to_baseunit = True)
import sys
import multiprocessing
import subprocess
import numpy as np
import shutil
import os

path_to_code = "../generated-codes/chapter7/"
sys.path.append(path_to_code)

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo

In [3]:
kB = cst.Boltzmann*ureg.J/ureg.kelvin # boltzman constant
Na = cst.Avogadro/ureg.mole # avogadro
R = kB*Na # gas constant

In [4]:
tau = 1
epsilon = (119.76*ureg.kelvin*kB*Na).to(ureg.kcal/ureg.mol) # kcal/mol
r_star = 3.822*ureg.angstrom # angstrom
sigma = r_star / 2**(1/6) # angstrom
N_atom = 400 # no units
m_argon = 39.948*ureg.gram/ureg.mol
T = (55 * ureg.degC).to(ureg.degK) # 55°C
volume_star = r_star**3 * Na * 2**(-0.5)
cut_off = sigma*2.5
displace_mc = sigma/5 # angstrom
volume = N_atom*volume_star*tau/Na
box_size = volume**(1/3)
folder = "./"

In [5]:
# Minimization
em = MinimizeEnergy(maximum_steps=100,
                    thermo_period=10,
                    dumping_period=10,
    number_atoms=[N_atom],
    epsilon=[epsilon.magnitude], 
    sigma=[sigma.magnitude],
    atom_mass=[m_argon.magnitude],
    box_dimensions=[box_size.magnitude,
                    box_size.magnitude,
                    box_size.magnitude],
    cut_off=cut_off.magnitude,
    data_folder=folder,
)
em.run()

# Equilibration run
mc1 = MonteCarlo(maximum_steps=20000,
    dumping_period=1000,
    thermo_period=1000,
    neighbor=50,
    displace_mc = displace_mc.magnitude,
    desired_temperature = T.magnitude,
    number_atoms=[N_atom],
    epsilon=[epsilon.magnitude], 
    sigma=[sigma.magnitude],
    atom_mass=[m_argon.magnitude],
    box_dimensions=[box_size.magnitude,
                    box_size.magnitude,
                    box_size.magnitude],
    initial_positions = em.atoms_positions*em.reference_distance,
    cut_off=cut_off.magnitude,
    data_folder=folder,
)
mc1.run()

step epot maxF
0 59895733968519.000 9619860237625140.000
10 160614186.280 513067747.593
20 92443.851 27535.614
30 7524.343 747.580
40 636.161 62.659
50 -276.795 142.632
60 -434.015 54.631
70 -518.141 52.242
80 -570.395 27.263
90 -599.030 1.358
100 -618.720 0.670
step  N     T (K)     p (atm)   V (A3)    Ep (kcal/mol) Ek (kcal/mol) dens (g/cm3) 
0     400   3.28e+02  2.94e+03  1.58e+04  -6.18e+02     0.0           1.68         
1000  400   3.28e+02  5.7e+03   1.58e+04  -4.91e+02     0.0           1.68         
2000  400   3.28e+02  6.54e+03  1.58e+04  -4.5e+02      0.0           1.68         
3000  400   3.28e+02  7.15e+03  1.58e+04  -4.23e+02     0.0           1.68         
4000  400   3.28e+02  7.25e+03  1.58e+04  -4.19e+02     0.0           1.68         
5000  400   3.28e+02  7.41e+03  1.58e+04  -4.13e+02     0.0           1.68         
6000  400   3.28e+02  7.63e+03  1.58e+04  -4.04e+02     0.0           1.68         
7000  400   3.28e+02  7.73e+03  1.58e+04  -4e+02        0.0      

In [7]:
# Short production run
mc2 = MonteCarlo(maximum_steps=5000,
    dumping_period=10,
    thermo_period=10,
    neighbor=50,
    displace_mc = displace_mc.magnitude,
    desired_temperature = T.magnitude,
    number_atoms=[N_atom],
    epsilon=[epsilon.magnitude], 
    sigma=[sigma.magnitude],
    atom_mass=[m_argon.magnitude],
    box_dimensions=[box_size.magnitude,
                    box_size.magnitude,
                    box_size.magnitude],
    initial_positions = mc1.atoms_positions*mc1.reference_distance,
        cut_off=cut_off.magnitude,
        data_folder=folder,
    )
mc2.run()

step  N     T (K)     p (atm)   V (A3)    Ep (kcal/mol) Ek (kcal/mol) dens (g/cm3) 
0     400   3.28e+02  8.71e+03  1.58e+04  -3.52e+02     0.0           1.68         
10    400   3.28e+02  8.69e+03  1.58e+04  -3.53e+02     0.0           1.68         
20    400   3.28e+02  8.72e+03  1.58e+04  -3.51e+02     0.0           1.68         
30    400   3.28e+02  8.74e+03  1.58e+04  -3.51e+02     0.0           1.68         
40    400   3.28e+02  8.73e+03  1.58e+04  -3.51e+02     0.0           1.68         
50    400   3.28e+02  8.77e+03  1.58e+04  -3.49e+02     0.0           1.68         
60    400   3.28e+02  8.79e+03  1.58e+04  -3.48e+02     0.0           1.68         
70    400   3.28e+02  8.8e+03   1.58e+04  -3.48e+02     0.0           1.68         
80    400   3.28e+02  8.76e+03  1.58e+04  -3.5e+02      0.0           1.68         
90    400   3.28e+02  8.72e+03  1.58e+04  -3.52e+02     0.0           1.68         
100   400   3.28e+02  8.67e+03  1.58e+04  -3.54e+02     0.0           1.68  